In [57]:
import pandas as ps
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

In [5]:
df = ps.read_csv("heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [8]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [13]:
df.groupby(['ChestPainType']).size()

ChestPainType
ASY    496
ATA    173
NAP    203
TA      46
dtype: int64

In [15]:
df.groupby('RestingECG').size()

RestingECG
LVH       188
Normal    552
ST        178
dtype: int64

In [17]:
df.groupby('ExerciseAngina').size()

ExerciseAngina
N    547
Y    371
dtype: int64

In [19]:
df.groupby('ST_Slope').size()

ST_Slope
Down     63
Flat    460
Up      395
dtype: int64

In [20]:
label = LabelEncoder()

In [25]:
df['n_ChestPainType'] = label.fit_transform(df['ChestPainType'])
df['n_RestingECG'] = label.fit_transform(df['RestingECG'])
df['n_ExerciseAngina'] = label.fit_transform(df['ExerciseAngina'])
df['n_ST_Slope'] = label.fit_transform(df['ST_Slope'])
df['n_Sex'] = label.fit_transform(df['Sex'])
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease,n_ChestPainType,n_RestingECG,n_ExerciseAngina,n_ST_Slope,n_Sex
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0,1,1,0,2,1
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1,2,1,0,1,0
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0,1,2,0,2,1
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1,0,1,1,1,0
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0,2,1,0,2,1


In [38]:
final=df.drop(['RestingECG','ST_Slope','ExerciseAngina','ChestPainType','Sex'],axis='columns')
final

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,n_ChestPainType,n_RestingECG,n_ExerciseAngina,n_ST_Slope,n_Sex
0,40,140,289,0,172,0.0,0,1,1,0,2,1
1,49,160,180,0,156,1.0,1,2,1,0,1,0
2,37,130,283,0,98,0.0,0,1,2,0,2,1
3,48,138,214,0,108,1.5,1,0,1,1,1,0
4,54,150,195,0,122,0.0,0,2,1,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,110,264,0,132,1.2,1,3,1,0,1,1
914,68,144,193,1,141,3.4,1,0,1,0,1,1
915,57,130,131,0,115,1.2,1,0,1,1,1,1
916,57,130,236,0,174,0.0,1,1,0,0,1,0


In [39]:
final.groupby(['n_ChestPainType']).size()

n_ChestPainType
0    496
1    173
2    203
3     46
dtype: int64

In [41]:
x = final.drop('HeartDisease',axis='columns')
y = final.HeartDisease


In [43]:
Scaled = StandardScaler()
x_scaled = Scaled.fit_transform(x)
x_scaled

array([[-1.4331398 ,  0.41090889,  0.82507026, ..., -0.8235563 ,
         1.05211381,  0.51595242],
       [-0.47848359,  1.49175234, -0.17196105, ..., -0.8235563 ,
        -0.59607813, -1.93816322],
       [-1.75135854, -0.12951283,  0.7701878 , ..., -0.8235563 ,
         1.05211381,  0.51595242],
       ...,
       [ 0.37009972, -0.12951283, -0.62016778, ...,  1.21424608,
        -0.59607813,  0.51595242],
       [ 0.37009972, -0.12951283,  0.34027522, ..., -0.8235563 ,
        -0.59607813, -1.93816322],
       [-1.64528563,  0.30282455, -0.21769643, ..., -0.8235563 ,
         1.05211381,  0.51595242]])

In [45]:
x_train,x_test,y_train,y_test = train_test_split(x_scaled,y,test_size=0.20,random_state=10)

In [47]:
s = SVC(kernel='linear')
s.fit(x_train,y_train)
s.score(x_test,y_test)

0.8478260869565217

In [60]:
bag1 = BaggingClassifier(estimator=SVC(),n_estimators=100,max_samples=0.8,oob_score=True,random_state=0)
bag1.fit(x_train,y_train)
bag1.oob_score_

0.8664850136239782

In [70]:
li = [5,6,7,8,9]
for i in li:
    print(max(cross_val_score(bag1,x,y,cv=i)))

0.7934782608695652
0.7777777777777778
0.8091603053435115
0.8173913043478261
0.8431372549019608


In [55]:
tree1 = DecisionTreeClassifier()
tree1.fit(x_train,y_train)
tree1.score(x_test,y_test)

0.7663043478260869

In [58]:
bag2 = BaggingClassifier(estimator=DecisionTreeClassifier(),n_estimators=100,max_samples=0.8,oob_score=True,random_state=0)
bag2.fit(x_train,y_train)
bag2.oob_score_

0.8501362397820164

In [69]:
li = [5,6,7,8,9]
for i in li:
    print(max(cross_val_score(bag2,x,y,cv=i)))

0.8369565217391305
0.8823529411764706
0.916030534351145
0.9391304347826087
0.9509803921568627
